# Lernen durch Gradientenabstieg 
Wir betrachten eine einzelnes künstliches Neuron mit zwei Eingängen, also drei Parametern $w_1,w_2,b$. Aufgrund der relativ kleinen Anzahl an Paramtern bleibt das Problem realtiv überschaubar.

## Vorbereiten der Daten

In [ ]:
# Lege fest, ob das Notebook in der lite-Version asugeführt wird
# (Bei lokaler Ausführung müssen einige Zeilen geringfügig angepasst werden)
run_in_lite_version = True

In [ ]:
# nur für jupyter lite notwendig
%pip install -q plotly 
from IPython.display import display, HTML

In [ ]:
# Import packages
import pandas as pd
import plotly.express as px

In [ ]:
# Lese Daten ein
df_tiere = pd.read_csv('tiere_gefaehrlichkeit.csv', sep=';')
df_tiere.head()

In [ ]:
# Visualisiere Daten
fig = px.scatter(df_tiere, x="Zahngröße", y="Augengröße",color="Gefahr",height=350, width=400)
if run_in_lite_version:
    display(HTML(fig.to_html())) # jupyter lite
else:
    fig.show() # jupyter notebook lokal

In [ ]:
# Erzeuge numerische Daten
df_tiere_num = df_tiere.replace(['ungefährlich','gefährlich'],[-1,1])
df_tiere_num.head()

# Aktivierungsfunktion
Für die Perzeptronen hatten wir die Treppenfunktion als Aktivierungsfunktion verwendet. Diese ersezten wir nun durch die die Funktion tanh.
$$ tanh(z) = \frac{e^z-e^{-z}}{e^z+e^{-z}} $$

In [ ]:
from numpy import tanh

In [ ]:
activation = lambda z: tanh(z)

In [ ]:
# Visualisiere die tanh-Funktion
import numpy as np
import matplotlib.pyplot as plt

# Generate x values from -5 to 5
x = np.linspace(-5, 5, 100)

# Calculate tanh values for each x
y = activation(x)

# Plot the tanh function
plt.figure(figsize=(6, 2))
plt.plot(x, y, label='tanh(x)')
plt.xlabel('x')
plt.ylabel('tanh(x)')
plt.title('Plot of tanh(x)')
plt.grid(True)
plt.legend()
plt.show()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 1:
Vergleiche die tanh-Funktion mit der früher verwendeten Treppenfunktion. Worin bestehen Ähnlichkeiten, was sind Unterschiede?

Lösung bitte hierher

## Künstliches Neuron
Weiterhin führen wir noch den so genannten Bias $b$ als negativen Schwellenwert ein, also $b := -\Theta$. Der Grund hierfür ist, dass dann im Weiteren die Formeln für die partiellen Ableitungen einfacher werden, weil sie weniger Minuszeichen enthalten.

Ein **künstliches Neuron** unterscheidet sich also von einem Perzeptron
- durch eine differenzierbare Aktivierungsfunktion.
- durch die Verwendung eines Bias´ statt eines Schwellenwertes (dies allerdings nur, um die Formeln einfacher zu machen).

In [ ]:
# Festlegung der Parameter des künstlichen Neurons
w1 = 1
w2 = 1
b = -1 

In [ ]:
# Funktion für den Output des KNN 
def neuron_output(x1, x2): 
    z = w1*x1+w2*x2+b
    a = activation(z)
    return a

In [ ]:
# Teste das künstliche Neuron
neuron_output(0,1)

# Heatmap

In [ ]:
# Zum Visualisieren des künstlichen Neurons
def plot_heatmap(x_min=0, x_max=4, y_min=0, y_max=4,resolution = 50):
    delta_x = x_max-x_min
    delta_y = y_max-y_min

    heatmap = pd.DataFrame(index = [y_min+i*delta_y/resolution for i in reversed(range(resolution+1))])

    for x in [x_min+i*delta_x/resolution for i in range(resolution+1)]:
        heatmap[x] = [neuron_output(x,y_min+i*delta_y/resolution) for i in reversed(range(resolution+1))]
    
    #display(heatmap)
    fig = px.imshow(heatmap, text_auto=False, color_continuous_scale="RdBu_r", height=300, width=300,\
                    origin='lower',zmin=-1,zmax=1,labels=dict(x="Zahngröße", y="Augengröße", color="Gefahr"))
    
    if run_in_lite_version:
        display(HTML(fig.to_html())) # jupter lite
    else:
        fig.show() # jupyter notebook lokal

In [ ]:
plot_heatmap()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 2:
Erläutere in Abgrenzung zum Perzeptron, woran man in der Heatmap erkennen kann, dass es sich nun um ein künstliches Neuron und nicht um ein Perzeptron handelt.

Erläuterung bitte hierher

## Funktion zum Testen der Vorhersage

In [ ]:
# Funktion zur Berechnung der Genauigkeit des Perzeptrons,
# gibt den Anteil der korrekt klassifizierten Tiere zurück
def accuracy():
    #Erstelle Spalte mit interpretierten Outputs des KNN
    Outputs=[]

    for i in range(len(df_tiere)):
        
        output = neuron_output(df_tiere.iloc[i]['Zahngröße'],df_tiere.iloc[i]['Augengröße'])
        
        if output > 0:
            Outputs.append('gefährlich')
        elif output <= 0:
            Outputs.append('ungefährlich')

    df_tiere['Vorhersagen'] = Outputs

    #display(df_tiere_test)#.head()
    return sum(df_tiere['Gefahr'] == df_tiere['Vorhersagen'])/len(df_tiere)

In [ ]:
accuracy()

In [ ]:
# Falsch Klassifizierte
df_tiere[df_tiere['Gefahr'] != df_tiere['Vorhersagen']]

In [ ]:
# Berechne wie viele Beispiele korrekt klassifiziert werden 
sum(df_tiere['Gefahr'] == df_tiere['Vorhersagen'])

# Lernprozess (Gradientenabstieg)

In [ ]:
lr = 0.05

In [ ]:
# Gradientenabstiegsverfahren
def gewichte_update(x1, x2, t, w1, w2, b):
    
    # Forward-Propagation
    # Berechnung der Neuronenaktivierung bis zum Output
    z = w1*x1+w2*x2+b  # Propagierungsfunktion
    a = tanh(z)        # Aktivierung
    
    # Backward-Progation
    # Aktualisierung der Gewichte
    w1 += lr*(t-a)*(1.0-a**2)*x1    
    w2 += lr*(t-a)*(1.0-a**2)*x2    
    b  += lr*(t-a)*(1.0-a**2)      

    return w1, w2, b 

# Einzelne Lern-Epoche

In [ ]:
# Lernrate
lr=0.5

In [ ]:
# Parameter des künstlichen Neurons
w1 = 1
w2 = 0.5
b  = -1

In [ ]:
print("Anteil korrekte Vorhersagen vor dem Training:  ", accuracy())

# Lernprozess: Aktualisieren der Parameter (Gewichte und des Bias)
for j in range( len(df_tiere_num) ):
    x1 = df_tiere_num.iloc[j]['Zahngröße']
    x2 = df_tiere_num.iloc[j]['Augengröße']
    t  = df_tiere_num.iloc[j]['Gefahr']

    w1, w2, b = gewichte_update(x1, x2, t, w1, w2, b)

print("Anteil korrekte Vorhersagen nach dem Training:  ", accuracy())

In [ ]:
plot_heatmap()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 3:
Führe die Lern-Regel (Gradientenabsteigsverfahren) für das künstliche Neuron für 10 Iterationen durch. Wie verändert sich die Genauigkeit der Vorhersage? Halte deine Ergebnisse schriftlich fest.

Ergebnisse bitte hierher

# Mehrere Epochen auf einmal

In [ ]:
def calc_epochs(n):
    global w1, w2, b
    for epoch in range(n):
        print("### Epoch ###", epoch, end=" ")
        #Aktualisieren der Gewichte 
        for j in range( len(df_tiere_num) ):
            x1 = df_tiere_num.iloc[j]['Zahngröße']
            x2 = df_tiere_num.iloc[j]['Augengröße']
            t  = df_tiere_num.iloc[j]['Gefahr']

            w1, w2, b = gewichte_update(x1, x2, t, w1, w2, b)
        print(w1,w2,b, end=" ")
        print("Accuracy: ", accuracy())
        if accuracy() == 1:
            return epoch

In [ ]:
lr=0.5

In [ ]:
#Festlegung der Kantengewichte
w1 = 1 
w2 = 0.5 
b = -1 

In [ ]:
print( "Number of epochs to 100% :", calc_epochs(100) )

In [ ]:
plot_heatmap()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 4:
Teste die Konvergenzrate für unterschiedliche Lernraten und Parameter künstlichen Neurons. Findest du eine Lernrate bzw. Paramter, für die das Verfahren nicht konvergiert?

Ergebnisse bitte hierher

## Fertiges Modell
Im fertigen Modell werden die Gewichte und der Schwellenwert zufällig vorbelegt.

In [ ]:
import random

In [ ]:
# Lernrate
lr = 0.5

In [ ]:
# Angepasst an die Eingangswerte im Bereich [0,4]x[0,4] 
# werden die Gewichte und der Bias vorbelegt mit nomalverteilten Zufallszahlen
w1 = random.normalvariate(1,1)
w2 = random.normalvariate(1,1)
b  = random.normalvariate(-1,1)
print("Initial parameters: ", w1, w2, b)

In [ ]:
plot_heatmap()

In [ ]:
print( "Number of epochs to 100% :", calc_epochs(100) )

In [ ]:
plot_heatmap()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 5:
Teste das fertige Modell für unterschiedliche Lernraten und Parameter des künstlichen Neurons. Was beobachtest du. Halte deine Ergenisse schriftlich fest.

Ergebnisse bitte hierher